In [1]:
import numpy as np

In [1]:
a = [43,44,45,422,966,967,970,1327,1328,1657,1658,1661,1662,1664,1667,1996,1999,2000]
a+= [1268,1269,1278,1461,1462,1466,1468,1515,1516,1520,1528,1593,1597,1598,1599,1781,1784,1785,1789,1850,1856,1857,1921,1925,1927,1930,2058,2062,2063,2066]

In [2]:
sorted(a)

[43,
 44,
 45,
 422,
 966,
 967,
 970,
 1268,
 1269,
 1278,
 1327,
 1328,
 1461,
 1462,
 1466,
 1468,
 1515,
 1516,
 1520,
 1528,
 1593,
 1597,
 1598,
 1599,
 1657,
 1658,
 1661,
 1662,
 1664,
 1667,
 1781,
 1784,
 1785,
 1789,
 1850,
 1856,
 1857,
 1921,
 1925,
 1927,
 1930,
 1996,
 1999,
 2000,
 2058,
 2062,
 2063,
 2066]